Compile OpenPose

In [2]:
!rm openpose -r
!rm cmake-3.13.0-Linux-x86_64.tar.gz

# Based on: https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/OpenPose.ipynb
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo
  
#!cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --write_json output/ --display 0 --render_pose 0

Selecting previously unselected package libgflags2.2.
(Reading database ... 145483 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

Run Pose Estimation:

In [ ]:
# Net resolutions: Optimal -> Less: 1312x736 (Out of Memory)
# Default: 16*82, 16*46
#   "1152x576" => 1/2 hours for 1:30 min video

video="exp2_room.avi"
output="./output/"

!rm -rv openpose.avi openpose/output/* 
!cd openpose && \
./build/examples/openpose/openpose.bin --net_resolution "1152x576" \
                                --scale_number 4 --scale_gap 0.25 \
                                --hand --hand_scale_number 6 --hand_scale_range 0.4 --face \
                                --video ../$video \
                                --write_video ../openpose.avi \
                                --write_json $output \
                                --number_people_max 1 \
                                --tracking 1 \
                                --display 0
# Convert the result to MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

removed 'openpose.avi'
removed 'openpose/output/exp1_room_000000000000_keypoints.json'
removed 'openpose/output/exp1_room_000000000001_keypoints.json'
removed 'openpose/output/exp1_room_000000000002_keypoints.json'
removed 'openpose/output/exp1_room_000000000003_keypoints.json'
removed 'openpose/output/exp1_room_000000000004_keypoints.json'
removed 'openpose/output/exp1_room_000000000005_keypoints.json'
removed 'openpose/output/exp1_room_000000000006_keypoints.json'
removed 'openpose/output/exp1_room_000000000007_keypoints.json'
removed 'openpose/output/exp1_room_000000000008_keypoints.json'
removed 'openpose/output/exp1_room_000000000009_keypoints.json'
removed 'openpose/output/exp1_room_000000000010_keypoints.json'
removed 'openpose/output/exp1_room_000000000011_keypoints.json'
removed 'openpose/output/exp1_room_000000000012_keypoints.json'
removed 'openpose/output/exp1_room_000000000013_keypoints.json'
removed 'openpose/output/exp1_room_000000000014_keypoints.json'
removed 'openpose

Saving Output:

In [ ]:
!rm output.zip
!7z a output.zip openpose/output/ output.mp4 -r

In [ ]:
!ls openpose/output/* | wc -l